# Init

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina' # High-resolution plot outputs for Retina laptops

In [2]:
# Batteries
import os
from datetime import datetime
from pathlib import Path


# Extrenals
import matplotlib.pyplot as plt
import numpy as np


# PySTEPS
from pysteps import io, motion, nowcasts
from pysteps.io import import_fmi_pgm
from pysteps.postprocessing import ensemblestats
from pysteps.utils import conversion, transformation

Pysteps configuration file found at: /Users/ilirium/.pysteps/pystepsrc



# Path and files

In [3]:
_path = Path('../pysteps-data')
path = _path / 'mrms/2019/06/10/'

In [4]:
!ls -lhg {path}

total 69488
-rw-r--r--  1 staff   1.0M May  7 11:56 PrecipRate_00.00_20190610-000000.grib2
-rw-r--r--  1 staff   1.0M May  7 11:56 PrecipRate_00.00_20190610-000200.grib2
-rw-r--r--  1 staff   1.0M May  7 11:56 PrecipRate_00.00_20190610-000400.grib2
-rw-r--r--  1 staff   999K May  7 11:56 PrecipRate_00.00_20190610-000600.grib2
-rw-r--r--  1 staff   996K May  7 11:56 PrecipRate_00.00_20190610-000800.grib2
-rw-r--r--  1 staff   994K May  7 11:56 PrecipRate_00.00_20190610-001000.grib2
-rw-r--r--  1 staff   990K May  7 11:56 PrecipRate_00.00_20190610-001200.grib2
-rw-r--r--  1 staff   992K May  7 11:56 PrecipRate_00.00_20190610-001400.grib2
-rw-r--r--  1 staff   991K May  7 11:56 PrecipRate_00.00_20190610-001600.grib2
-rw-r--r--  1 staff   985K May  7 11:56 PrecipRate_00.00_20190610-001800.grib2
-rw-r--r--  1 staff   982K May  7 11:56 PrecipRate_00.00_20190610-002000.grib2
-rw-r--r--  1 staff   982K May  7 11:56 PrecipRate_00.00_20190610-002200.grib2
-rw-r--r--  1 staff   979K May  7 11:56 

In [5]:
!ls {path} | wc -l

      36


# Functions

## `show_array_structure`

In [6]:
def show_array_structure(array):
    print(f'shape = {array.shape}')
    
    size = array.shape[0] * array.shape[1] * array.shape[2]
    print(f'shape[0] * shape[1] * shape[2] = {size}')
    
    nan_amount = array[np.isnan(array)].shape[0]
    nan_percent = round(nan_amount / size, 3)
    print(f'NAN amount = {nan_amount}, part = {nan_percent}')
    
    finite_amount = array[~np.isnan(array)].shape[0]
    finite_percent = round(finite_amount / size, 3)
    print(f'FINITE amount = {finite_amount}, part = {finite_percent}')
    print()
    print(f'unique = {np.sort(np.unique(array[np.isfinite(array)]))}')

# Read

In [7]:
date = datetime.strptime("20190610", "%Y%m%d")
root_path = "../pysteps-data/mrms/2019/06/10/"
fn_pattern = 'PrecipRate_00.00_%Y%m%d-%H%M00'
fn_ext = "grib2"

In [8]:
fns = io.archive.find_by_date(date, root_path, "", fn_pattern, fn_ext, 2, num_next_files=35)

In [9]:
Z, _, metadata = io.read_timeseries(fns, io.import_mrms_grib)

In [10]:
R = conversion.to_rainrate(Z, metadata, 223.0, 1.53)[0]
R = transformation.dB_transform(R, threshold=0.1, zerovalue=-15.0)[0]

In [11]:
metadata

{'xpixelsize': 0.04,
 'ypixelsize': 0.04,
 'unit': 'mm/h',
 'transform': None,
 'zerovalue': 0,
 'projection': {'proj': 'latlon', 'ellps': 'IAU76'},
 'yorigin': 'upper',
 'threshold': 0.0125,
 'x1': -129.98000000042865,
 'x2': -60.0200019995713,
 'y1': 20.020000999571305,
 'y2': 54.98000000042869,
 'timestamps': array([datetime.datetime(2019, 6, 10, 0, 0),
        datetime.datetime(2019, 6, 10, 0, 2),
        datetime.datetime(2019, 6, 10, 0, 4),
        datetime.datetime(2019, 6, 10, 0, 6),
        datetime.datetime(2019, 6, 10, 0, 8),
        datetime.datetime(2019, 6, 10, 0, 10),
        datetime.datetime(2019, 6, 10, 0, 12),
        datetime.datetime(2019, 6, 10, 0, 14),
        datetime.datetime(2019, 6, 10, 0, 16),
        datetime.datetime(2019, 6, 10, 0, 18),
        datetime.datetime(2019, 6, 10, 0, 20),
        datetime.datetime(2019, 6, 10, 0, 22),
        datetime.datetime(2019, 6, 10, 0, 24),
        datetime.datetime(2019, 6, 10, 0, 26),
        datetime.datetime(2019, 6,

In [12]:
show_array_structure(Z)

shape = (36, 875, 1750)
shape[0] * shape[1] * shape[2] = 55125000
NAN amount = 19810981, part = 0.359
FINITE amount = 35314019, part = 0.641

unique = [0.0000000e+00 6.2500001e-03 1.2500000e-02 ... 1.3475000e+02 1.3677499e+02
 1.4481876e+02]


In [13]:
show_array_structure(R)

shape = (36, 875, 1750)
shape[0] * shape[1] * shape[2] = 55125000
NAN amount = 19810981, part = 0.359
FINITE amount = 35314019, part = 0.641

unique = [-15.       -10.        -9.736711 ...  21.295288  21.360065  21.608248]
